<a href="https://colab.research.google.com/github/VrijKun/CS6910_Assignment_1/blob/main/Assignment_1_DL_ED23D015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.9 MB/s eta 0:00:00


In [ ]:
import wandb
wandb.login() #3c21150eb43b007ee446a1ff6e87f640ec7528c4

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from keras.datasets import fashion_mnist
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Load the Fashion-MNIST dataset
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()


image_classes = len(np.unique(Y_train)) #to find out number of unique images
image_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'] #taken from keras documentation

def show_images(image_classe,img_labels,X,Y):
   set_Labels=[]
   set_Images = []

   for i in range(len(X)):
     if len(set_Images)== image_classes:
       break
     if image_labels[Y[i]] not in set_Labels:

      set_Labels.append(image_labels[Y[i]])
      set_Images.append(X[i])

   run=wandb.init(project='DL_Assignment_1',entity='ed23d015',reinit=True)
   run.log({"Sample from each class":list(wandb.Image(img,caption= caption) for img,caption in zip(set_Images,set_Labels))})
   run.finish()

#Getting prints
show_images(image_classes,image_labels,X_test,Y_test)

wandb: Currently logged in as: ed23d015. Use `wandb login --relogin` to force relogin


In [ ]:

from keras.utils import to_categorical

def initialize_parameters(layer_dims):
    parameters = {}
    L = len(layer_dims)
    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
    return parameters

def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

def relu(Z):
    return np.maximum(0, Z)

def softmax(Z):
    exp_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)

def forward_propagation(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2
    for l in range(1, L):
        A_prev = A
        Z = np.dot(parameters['W' + str(l)], A_prev) + parameters['b' + str(l)]
        A = relu(Z)
        cache = (A_prev, parameters['W' + str(l)], parameters['b' + str(l)], Z)
        caches.append(cache)
    ZL = np.dot(parameters['W' + str(L)], A) + parameters['b' + str(L)]
    AL = softmax(ZL)
    cache = (A, parameters['W' + str(L)], parameters['b' + str(L)], ZL)
    caches.append(cache)
    return AL, caches

def compute_cost(AL, Y):
    m = Y.shape[1]
    cost = -np.sum(np.multiply(Y, np.log(AL))) / m
    return cost

def backward_propagation(AL, Y, caches):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    dZL = AL - Y
    A_prev, W, b, Z = caches[L-1]
    grads['dW' + str(L)] = np.dot(dZL, A_prev.T) / m
    grads['db' + str(L)] = np.sum(dZL, axis=1, keepdims=True) / m
    dAL = np.dot(W.T, dZL)
    for l in reversed(range(L-1)):
        A_prev, W, b, Z = caches[l]
        dZ = np.array(dAL, copy=True)
        dZ[Z <= 0] = 0
        grads['dW' + str(l + 1)] = np.dot(dZ, A_prev.T) / m
        grads['db' + str(l + 1)] = np.sum(dZ, axis=1, keepdims=True) / m
        dAL = np.dot(W.T, dZ)
    return grads

def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2
    for l in range(L):
        parameters['W' + str(l+1)] -= learning_rate * grads['dW' + str(l+1)]
        parameters['b' + str(l+1)] -= learning_rate * grads['db' + str(l+1)]
    return parameters

def predict(X, parameters):
    AL, _ = forward_propagation(X, parameters)
    return np.argmax(AL, axis=0)

def accuracy(predictions, labels):
    return np.mean(predictions == np.argmax(labels, axis=0))


# Preprocess the data
train_images = X_train / 255.0
test_images = X_test / 255.0
train_images = train_images.reshape(train_images.shape[0], -1).T
test_images = test_images.reshape(test_images.shape[0], -1).T

# Convert labels to one-hot encoding
train_labels = to_categorical(Y_train, num_classes=10).T
test_labels = to_categorical(Y_test, num_classes=10).T

# Define hyperparameters
layer_dims = [train_images.shape[0], 128, 10]
learning_rate = 0.01
num_epochs = 10000

# Initialize parameters
parameters = initialize_parameters(layer_dims)

# Train the model
for epoch in range(num_epochs):
    AL, caches = forward_propagation(train_images, parameters)
    cost = compute_cost(AL, train_labels)
    grads = backward_propagation(AL, train_labels, caches)
    parameters = update_parameters(parameters, grads, learning_rate)
    print(f"Epoch {epoch+1}/{num_epochs}, Cost: {cost:.4f}")

# Evaluate the model
train_predictions = predict(train_images, parameters)
test_predictions = predict(test_images, parameters)
train_acc = accuracy(train_predictions, train_labels)
test_acc = accuracy(test_predictions, test_labels)
print("Train Accuracy:", train_acc)
print("Test Accuracy:", test_acc)


Streaming output truncated to the last 5000 lines.
Epoch 5003/10000, Cost: 0.5127
Epoch 5004/10000, Cost: 0.5127
Epoch 5005/10000, Cost: 0.5127
Epoch 5006/10000, Cost: 0.5126
Epoch 5007/10000, Cost: 0.5126
Epoch 5008/10000, Cost: 0.5126
Epoch 5009/10000, Cost: 0.5126
Epoch 5010/10000, Cost: 0.5125
Epoch 5011/10000, Cost: 0.5125
Epoch 5012/10000, Cost: 0.5125
Epoch 5013/10000, Cost: 0.5124
Epoch 5014/10000, Cost: 0.5124
Epoch 5015/10000, Cost: 0.5124
Epoch 5016/10000, Cost: 0.5124
Epoch 5017/10000, Cost: 0.5123
Epoch 5018/10000, Cost: 0.5123
Epoch 5019/10000, Cost: 0.5123
Epoch 5020/10000, Cost: 0.5123
Epoch 5021/10000, Cost: 0.5122
Epoch 5022/10000, Cost: 0.5122
Epoch 5023/10000, Cost: 0.5122
Epoch 5024/10000, Cost: 0.5121
Epoch 5025/10000, Cost: 0.5121
Epoch 5026/10000, Cost: 0.5121
Epoch 5027/10000, Cost: 0.5121
Epoch 5028/10000, Cost: 0.5120
Epoch 5029/10000, Cost: 0.5120
Epoch 5030/10000, Cost: 0.5120
Epoch 5031/10000, Cost: 0.5120
Epoch 5032/10000, Cost: 0.5119
Epoch 5033/10000, C